In [1]:
import numpy as np 
import pandas as pd 

df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

print(df_train.columns)
print("shape: ", df_train.shape)

Index(['id', 'Product ID', 'Type', 'Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF',
       'RNF'],
      dtype='object')
shape:  (136429, 14)


# Analysing the dataset

In [2]:
print("non-zero:", np.count_nonzero(df_train["TWF"]), "\nunique elements:", df_train["TWF"].unique())
print("non-zero:", np.count_nonzero(df_train["HDF"]), "\nunique elements:", df_train["HDF"].unique())
print("non-zero:", np.count_nonzero(df_train["PWF"]), "\nunique elements:", df_train["PWF"].unique())
print("non-zero:", np.count_nonzero(df_train["OSF"]), "\nunique elements:", df_train["OSF"].unique())
print("non-zero:", np.count_nonzero(df_train["RNF"]), "\nunique elements:", df_train["RNF"].unique())

non-zero: 212 
unique elements: [0 1]
non-zero: 704 
unique elements: [0 1]
non-zero: 327 
unique elements: [0 1]
non-zero: 540 
unique elements: [0 1]
non-zero: 308 
unique elements: [0 1]


In [3]:
import matplotlib.pyplot as plt 

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(10,6))

axes[0, 0].hist(df_train["Air temperature [K]"], bins=50)
axes[0, 1].hist(df_train["Process temperature [K]"], bins=50)
axes[0, 2].hist(df_train["Rotational speed [rpm]"], bins=50)
axes[1, 0].hist(df_train["Torque [Nm]"], bins=50)
axes[1, 1].hist(df_train["Tool wear [min]"], bins=50)

axes[0, 0].set_title("Air Temperature [K]")
axes[0, 1].set_title("Process temperature [K]")
axes[0, 2].set_title("Rotational speed [rpm]")
axes[1, 0].set_title("Torque [Nm]")
axes[1, 1].set_title("Tool wear [min]")

plt.tight_layout()
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [8]:
from sklearn.preprocessing import StandardScaler

x_train = df_train.copy()
x_train.drop('id', axis=1, inplace=True)
x_train.drop('Product ID', axis=1, inplace=True)  # deal with this later
x_train.drop('Machine failure', axis=1, inplace=True)
y_train = df_train["Machine failure"]

x_test = df_test.copy()
x_test.drop('id', axis=1, inplace=True)
x_test.drop('Product ID', axis=1, inplace=True)  # deal with this later

# Standardization
columns_to_standardize = ['Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]']

scaler = StandardScaler()

x_train[columns_to_standardize] = scaler.fit_transform(x_train[columns_to_standardize])
x_test[columns_to_standardize] = scaler.fit_transform(x_test[columns_to_standardize])

x_train = np.array(x_train)
y_train = np.array(y_train, dtype=np.float32)
x_test = np.array(x_test)

lm_column = 0

mapping = {'L': 0.0, 'M': 0.5, 'H': 1.0}
x_train[:, lm_column] = np.where(x_train[:, lm_column] == 'L', 0.0, np.where(x_train[:, lm_column] == 'M', 0.5, 1.0))
x_test[:, lm_column] = np.where(x_test[:, lm_column] == 'L', 0.0, np.where(x_test[:, lm_column] == 'M', 0.5, 1.0))

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

print(x_train[:5, :])

[[ 0.          0.3958803  -0.24623038  0.54541594 -0.49971116  0.55641687
   0.          0.          0.          0.          0.        ]
 [ 0.5         1.4698555   1.5586048   1.7203082  -1.3230277   1.4944327
   0.          0.          0.          0.          0.        ]
 [ 0.         -0.30220363 -1.0403578   2.051873   -1.6288309  -1.2414467
   0.          0.          0.          0.          0.        ]
 [ 0.          0.61067533  0.6922839   0.0264451   0.46474534  1.4475319
   0.          0.          0.          0.          0.        ]
 [ 0.5        -1.0002875  -0.67939085  0.8697727  -0.5820428  -1.1007444
   0.          0.          0.          0.          0.        ]]


In [ ]:
print(x_train.shape)

In [9]:
import tensorflow as tf
from tensorflow import keras

num_features = x_train.shape[1] 

model = keras.Sequential([
    keras.layers.Dense(11, activation='relu', input_shape=(num_features,)),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, batch_size=32)


2023-06-26 10:24:38.480907: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-26 10:24:38.526154: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-26 10:24:38.527059: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-26 10:24:39.815962: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/5


2023-06-26 10:24:44.121817: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


4264/4264 [==============================] - 6s 1ms/step - loss: 0.0698 - accuracy: 0.9842
Epoch 2/5
4264/4264 [==============================] - 5s 1ms/step - loss: 0.0233 - accuracy: 0.9959
Epoch 3/5
4264/4264 [==============================] - 5s 1ms/step - loss: 0.0222 - accuracy: 0.9962
Epoch 4/5
4264/4264 [==============================] - 5s 1ms/step - loss: 0.0220 - accuracy: 0.9962
Epoch 5/5
4264/4264 [==============================] - 5s 1ms/step - loss: 0.0219 - accuracy: 0.9962


In [10]:
predictions = model.predict(x_test)

2843/2843 [==============================] - 3s 909us/step


In [11]:
plt.hist(predictions, bins=100)
plt.show()

NameError: name 'plt' is not defined

In [12]:
print("y_train")
print(np.count_nonzero(y_train), "of", y_train.shape[0], "===", np.count_nonzero(y_train) / y_train.shape[0])

print("predictions")
print(np.count_nonzero(predictions > 0.5), "of", predictions.shape[0], "===", np.count_nonzero(predictions > 0.5) / predictions.shape[0])

predictions

y_train
2148 of 136429 === 0.01574445315878589
predictions
1112 of 90954 === 0.012225960375574467


array([[0.00093165],
       [0.00106039],
       [0.0010786 ],
       ...,
       [0.00334553],
       [0.00095841],
       [0.00044161]], dtype=float32)

In [13]:
indices = np.array(df_test["id"], dtype=int).reshape(-1, 1)

submission = np.hstack((indices, predictions))

submission_df = pd.DataFrame(submission, columns=['id', 'Machine failure'])
submission_df['id'] = submission_df['id'].astype(int)  # Convert "id" column to integer
submission_df.to_csv("submissions/supervised_learning_late.csv", index=False)

In [ ]:
indices = np.array(df_test["id"], dtype=int).reshape(-1, 1)
predictions = np.where(predictions < 10, 0, 1)
submission = np.hstack((indices, predictions))

submission_df = pd.DataFrame(submission, columns=['id', 'Machine failure'])
submission_df['id'] = submission_df['id'].astype(int)  # Convert "id" column to integer
submission_df.to_csv("submissions/troll.csv", index=False)